# Pipple Lecture 5: Gradient Boosting met XGBoost

### Case
Omdat Bart bijna jarig is, is Pippi op zoek naar een geschikt cadeau voor hem. 
Pippi weet dat Bart van een biertje houdt en heeft dit jaar bij verschillende bieren bijgehouden hoe Bart hierop reageert.
Die reactie heeft Pippi getransformeerd tot een rating van 0 tot 10.

Om te kijken welk nieuw biertje het beste past bij Bart zijn smaak, wil Pippi een model maken om deze score te voorspellen op basis van verschillende eigenschappen van bieren. Hiervoor wil Pippi natuurlijk het beste model hebben, om het beste model te krijgen wil Pippi gebruik maken van de huidige Gradient Boosting en met name het package XGBoost. Het beste model zal gebaseerd wordt gemeten op basis van de Mean Absolute Error (MAE).

###  Importeren van packages

We gebruiken onderstaande packages. Mocht je deze nog niet hebben, dan zijn deze te downloaden via:

- pip install xgboost
- pip install sklearn

In [ ]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

### Inlezen van data
We gebruiken de volgende datasets:
- **train.csv** om te trainen
- **valid.csv** om te checken hoe goed ons model werkt
- **test.csv** om de definitieve voorspellingen te maken

We hebben de volgende eigenschappen van de verschillende bieren:
- **calorien**: hoeveel calorien zitten er in het bier?
- **dichtheid**: de dichtheid van het bier?
- **is_belgisch**: of het bier belgisch is?
- **is_speciaal**: of het een speciaal bier is of een gewoon bier?
- **pct_alcohol**: wat is het percentage alcohol in het bier?
- **pct_eiwitten**: wat is het percentage eiwitten in het bier?
- **pct_gist**: wat is het percentage gist in het bier?
- **pct_hop**: wat is het percentage hop in het bier?
- **recensie**: wat is de gemiddelde recensie van het publiek?
- **suiker**: hoeveel suiker zit er in het bier?
- **water_kw**: wat is de water kwaliteit van het bier?
- **zuur**: hoe zuur is het bier?

Daarnaast is er voor elk bier een **beoordeling** van Bart op een schaal van 0-10. Dit is de waarde die je gaat voorspellen. Aan de hand van deze data willen we een model maken die inschat hoe Bart andere bieren beoordeelt, zodat het beste bier voor zijn verjaardag gekozen kan worden. De data is zover als mogelijk als geprepocessed, zodat hier geen tijd meer aan gespendeerd hoeft te worden!

In [ ]:
# Training features en labels
url_train = 'https://raw.githubusercontent.com/PippleNL/gradient-boosting-lecture/master/data/train.csv'
train = pd.read_csv(url_train)
train_X = train.iloc[:,:-1].copy()
train_y = train.iloc[:,-1:].copy()

# Validatie features en labels
url_valid = 'https://raw.githubusercontent.com/PippleNL/gradient-boosting-lecture/master/data/valid.csv'
valid = pd.read_csv(url_valid)
valid_X = valid.iloc[:,:-1].copy()
valid_y = valid.iloc[:,-1:].copy()

# Test features en labels
url_test_X = 'https://raw.githubusercontent.com/PippleNL/gradient-boosting-lecture/master/data/test_X.csv'
test_X = pd.read_csv(url_test_X)

### De XGBoost data structure
XGBoost maakt gebruik van een speciale data structure, waardoor XGBoost efficienter en effectiever kan werken. Mede hieraan dankt XGBoost zijn hoge performance.De representatie van XGBoost heet een DMatrix. In onderstaande code wordt de data omgezet naar deze representatie:

In [ ]:
dtrain = xgb.DMatrix(data=train_X.values, label=train_y.values, feature_names=train_X.columns)
dvalid = xgb.DMatrix(data=valid_X.values, label=valid_y.values, feature_names=valid_X.columns)
dtest = xgb.DMatrix(data=test_X.values, feature_names=test_X.columns)

### Een XGBoost model

XGBoost heeft verscheidene parameters die getuned kunnen worden, zodat het model beter presteert.
De volgende parameters kunnen getuned worden:
- **eta** staat voor hoeveel elke decision tree bijdraagt aan de uiteindelijke voorspelling [default=0.3]
- **gamm** is de minimale toegevoegde waarde van een nieuwe split [default=0]
- **max_depth**	is de maximale diepte van elke decision tree [default=6]
- ***subsample** is de ratio van random training samples waarop elke decision tree wordt getraind [default=1.0]
- **colsample_bytree** en **colsample_bylevel** geven aan welke ratios van de kolommen worden gebruikt per tree en per level [default=1.0]
- **objective** is het type loss functie die we minimalizeren, bijv. binary:logistic, multi:softmax, multi:softprob [default=reg:linear]
- **eval_metric** is de metric waarmee we evalueren hoe goed onze voorspellingen zijn op de validatie data [default according to objective]
- **seed** staat voor welke random seed we gebruiken [default=0]
- **num_round** geeft aan hoeveel decision trees we trainen

Als voorbeeld geven we een simpel model, waarbij we enkele parameters kiezen:

In [ ]:
# Initialiseren van de parameters
# Deze kun je allemaal aanpassen
param = {'eta':0.5, 
         'max_depth':3, 
         'min_child_weight':1,
         'gamma':0,
         'subsample':1.0,
         'colsample_bytree':1.0,
         'colsample_bylevel':1.0,
         'silent':1, 
         'objective':'reg:linear', 
         'eval_metric':'mae',
         'seed': 42}

num_round = 5

In [ ]:
# Het model wordt getraind, gegeven de parameters
bst = xgb.train(param, dtrain, num_round)

# Maak voorspellingen op training en validatie data
y_true_train = train_y.values
y_pred_train = bst.predict(dtrain)
y_true_valid = valid_y.values
y_pred_valid = bst.predict(dvalid)

# Bereken de MAE score
train_mae = mean_absolute_error(y_true_train, y_pred_train)
valid_mae = mean_absolute_error(y_true_valid, y_pred_valid)
print("Train MAE: {}".format(round(train_mae, 5)))
print("Valid MAE: {}".format(round(valid_mae, 5)))

### Verbeteringen

In het huidige model hebben we enkele parameters gekozen, maar hier is niet voldoende over nagedacht.
Daarnaast hebben we dataset opgesplitst in een train en validatie set. Echter is het ook mogelijk om cross validation te doen binnen XGBoost, waardoor het model mogelijk verbeterd kan worden.

Enkele tips zijn dus:
- Maak gebruik van cross-validation
- Verander de hyper parameters gegeven in bovenstaand stuk (misschien kun deze zelfs optimaliseren)
- Check of het model niet resulteert in overfitting
- maak gebruik van een Tree based model of een Lineair model

### Cross validation
In dit stuk wordt uitgelegd hoe gewerkt kan worden met [cross-validation](https://www.analyticsvidhya.com/blog/2018/05/improve-model-performance-cross-validation-in-python-r/). Om gebruik te maken van cross-validation dien je [XGBoost.cv()](https://rdrr.io/cran/xgboost/man/xgb.cv.html)
te gebruiken. Hiervoor kan je de training en validatie data weer samenvoegen.

In [ ]:
# Voeg train en valid data samen
combi_X = pd.concat([train_X, valid_X], ignore_index=True)
combi_y = pd.concat([train_y, valid_y], ignore_index=True)

# Maak een DMatrix aan
dcombi = xgb.DMatrix(data=combi_X.values, label=combi_y.values, feature_names=combi_X.columns)

In [ ]:
# Gebruik CV om overfitting te voorkomen
param = {'eta':0.5, 
         'max_depth':3, 
         'min_child_weight':1,
         'gamma':0,
         'subsample':1.0,
         'colsample_bytree':1.0,
         'colsample_bylevel':1.0,
         'silent':1, 
         'objective':'reg:linear', 
         'eval_metric':'mae',
         'seed': 42}

num_round = 5

cv_results = xgb.cv(param, dcombi, 
                    num_boost_round=num_round,
                    nfold=5, 
                    metrics={'mae'}, 
                    early_stopping_rounds=10)

cv_results.tail()

### Parameter tuning met Grid Search
We kunnen ook automatisch zoeken naar de optimale parameters voor ons model. Hiervoor definiëren we per parameter een range aan mogelijkheden. Daarna probeert de grid search alle mogelijke combinaties uit. Let op: als je veel mogelijkheden invoert per parameter kan de grid search lang duren! Probeer daarom eerst handmatig gevoel te krijgen voor welke parameters goed werken, zodat je ze daarna kan fine tunen met de grid search. Let op: de waarden van MAE die uit de grid search komen zijn negatief. Dit is omdat de sklearn API de metric altijd probeert te maximaliseren. Door de absolute waardes te nemen krijg je de echte MAE. Hoe dichter bij de 0, des te beter je model voorspelt.

In [ ]:
# We timen deze cel om een indicatie te geven van hoe lang de grid search duurt
start = time.time()

# Definieer hier je parameters
# Voeg vooral meer opties toe
cv_params = {'eta':[0.3, 0.5], 
             'n_estimators': [10, 20],
             'max_depth':[6, 10],
             'subsample':[0.8, 1.0],
             'colsample_bytree':[0.8, 1.0],
             'colsample_bylevel':[0.8, 1.0]}

# Deze parameters hoef je niet aan te passen
ind_params = {'seed':42, 
              'objective':'reg:linear',
              'eval_metric':'mae',
              'silent':1}

# Initialiseer de grid search
optimized_bst = GridSearchCV(xgb.XGBRegressor(**ind_params), 
                             cv_params, 
                             scoring = 'neg_mean_absolute_error', 
                             cv = 5, 
                             n_jobs = -1) 
optimized_bst.fit(valid_X, valid_y)

end = time.time()
print('Grid search voltooid in {} seconden'.format(round(end - start, 2)))

In [ ]:
# Laat de beste combinatie van parameters zien
optimized_bst.best_params_

### Meerdere voorspellingen combineren
Met de onderstaande cel kun je meerdere voorspellingen combineren. Momenteel wordt de unweighted average berekend. Je kunt zelf weights toevoegen mocht je dat willen.

In [ ]:
# Selecteer de lijsten met voorspellingen
preds_a = [1, 2, 3]
preds_b = [4, 5, 6]
preds_c = [7, 8, 9]

# Bereken het gemiddelde van de lijsten
combi_lists = np.array([preds_a,
                        preds_b,
                        preds_c])

combi_preds = list(np.average(combi_lists, axis=0))

combi_preds

### Voorspellingen maken op de test set
Met de onderstaande cel kun je voorspellingen maken op de test set data. Deze worden opgeslagen als .csv bestand als '/data/preds-naam-van-team.csv'. Verander 'naam-van-team' naar jullie team naam en stuur het bestand naar gijs@pipple.nl.

In [ ]:
# Maak nieuwe voorspellingen
preds = bst.predict(dtest)

# Voer hier de naam van je team in
naam = 'naam-van-team'

# Sla output op als .csv
path = 'output-{}.csv'.format(naam)
df = pd.DataFrame({'output': preds})
df.to_csv(path, sep=',',index=False)

from google.colab import files
files.download(path)